In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import os
import pandas as pd

In [2]:
finished_df = pd.read_csv('data_combined.csv', index_col=0)
finished_df

,Topic,Description,Title,vid_url,normalized_annotation,Num_of_Views,Likes,Dislikes,Number_of_Comments,Video_ID,Captions
0,chemtrails,Texts between Sean Hannity and convicted Trump...,A Judge Releases Private Texts Between Sean Ha...,https://www.youtube.com/watch?v=l8laDBhw6kA,0,510265,4436,681,2532,l8laDBhw6kA,>>> WHY DO YOU THINK NANCY >>> WHY DO YOU THIN...
1,chemtrails,INSTAGRAM FLYWITHCAPTAINJOE: https://goo.gl/TT...,COCKPIT WINDOWS - How to open SLIDING WINDOW a...,https://www.youtube.com/watch?v=XuMtzqtjeEM,0,775342,17572,217,1130,XuMtzqtjeEM,Dear friends and followers welcome back to my ...
2,chemtrails,Mystical BS like this drives me crazy! Living...,living without eating for NINE YEARS' BUSTED,https://www.youtube.com/watch?v=iwOX7vOf0_s,0,2104223,46140,2766,18920,iwOX7vOf0_s,the independent brevarian couple claimed they ...
3,chemtrails,"In part three of Hello World Shenzhen, Bloombe...",Inside China's High-Tech Dystopia,https://www.youtube.com/watch?v=ydPqKhgh9Mg,0,2129195,32969,6255,6828,ydPqKhgh9Mg,There's no disputing that Shenzhen has become ...
4,chemtrails,They claim to have 10x the energy density of a...,New 'Perfect' Battery: BUSTED,https://www.youtube.com/watch?v=8RbwOhM6PUk,0,579669,16807,2344,3932,8RbwOhM6PUk,so while making my latest for the about seeing...
...,...,...,...,...,...,...,...,...,...,...,...
1734,vaccines,US: Anti-vaccination groups blamed in Minnesot...,US: Anti-vaccination groups blamed in Minnesot...,https://www.youtube.com/watch?v=U8K9bbMo-M4,-1,6475,71,29,91,U8K9bbMo-M4,at this Minneapolis Medical Clinic signs in En...
1735,vaccines,"Dr. Anthony Fauci, director of the National In...",Anti-Vaccination: An Extreme Public Health Ris...,https://www.youtube.com/watch?v=duOLIS0Pkew,-1,2852,30,13,11,duOLIS0Pkew,>>> SHOULD PARENTS HAVE THEIR \n CHILDREN VA...
1736,vaccines,Parents in one Oregon town share with CNN's Ga...,Parent: 'Too many' vaccinations recommended fo...,https://www.youtube.com/watch?v=u8lVmtYHNL4,0,10883,72,67,331,u8lVmtYHNL4,RECEIVE AND WHEN THEY SHOULD GET THEM. >> Rep...
1737,vaccines,What is the role of gov't?\nWatch Andrea Tanta...,Debate heats up over vaccine exemptions,https://www.youtube.com/watch?v=_tc6lD6XTaI,0,2083,14,5,7,_tc6lD6XTaI,starting today kids in Oregon can be sent home...


In [3]:
finished_df['normalized_annotation'] = finished_df['normalized_annotation'].apply(pd.to_numeric, errors = 'coerce').fillna(1)
finished_df['normalized_annotation']

0       0
1       0
2       0
3       0
4       0
       ..
1734   -1
1735   -1
1736    0
1737    0
1738   -1
Name: normalized_annotation, Length: 1739, dtype: int64

In [4]:
finished_df['normalized_annotation'] = finished_df['normalized_annotation'] + 1

In [5]:
finished_df['Captions'] = finished_df['Captions'].astype('string')
finished_df = finished_df.dropna()
finished_df['Captions']

0       >>> WHY DO YOU THINK NANCY >>> WHY DO YOU THIN...
1       Dear friends and followers welcome back to my ...
2       the independent brevarian couple claimed they ...
3       There's no disputing that Shenzhen has become ...
4       so while making my latest for the about seeing...
                              ...                        
1734    at this Minneapolis Medical Clinic signs in En...
1735    >>> SHOULD PARENTS HAVE THEIR   
 CHILDREN VAC...
1736    RECEIVE AND WHEN THEY SHOULD GET THEM.  >> Rep...
1737    starting today kids in Oregon can be sent home...
1738    (lively upbeat music) - The effectiveness of t...
Name: Captions, Length: 1697, dtype: string

In [6]:
finished_df

,Topic,Description,Title,vid_url,normalized_annotation,Num_of_Views,Likes,Dislikes,Number_of_Comments,Video_ID,Captions
0,chemtrails,Texts between Sean Hannity and convicted Trump...,A Judge Releases Private Texts Between Sean Ha...,https://www.youtube.com/watch?v=l8laDBhw6kA,1,510265,4436,681,2532,l8laDBhw6kA,>>> WHY DO YOU THINK NANCY >>> WHY DO YOU THIN...
1,chemtrails,INSTAGRAM FLYWITHCAPTAINJOE: https://goo.gl/TT...,COCKPIT WINDOWS - How to open SLIDING WINDOW a...,https://www.youtube.com/watch?v=XuMtzqtjeEM,1,775342,17572,217,1130,XuMtzqtjeEM,Dear friends and followers welcome back to my ...
2,chemtrails,Mystical BS like this drives me crazy! Living...,living without eating for NINE YEARS' BUSTED,https://www.youtube.com/watch?v=iwOX7vOf0_s,1,2104223,46140,2766,18920,iwOX7vOf0_s,the independent brevarian couple claimed they ...
3,chemtrails,"In part three of Hello World Shenzhen, Bloombe...",Inside China's High-Tech Dystopia,https://www.youtube.com/watch?v=ydPqKhgh9Mg,1,2129195,32969,6255,6828,ydPqKhgh9Mg,There's no disputing that Shenzhen has become ...
4,chemtrails,They claim to have 10x the energy density of a...,New 'Perfect' Battery: BUSTED,https://www.youtube.com/watch?v=8RbwOhM6PUk,1,579669,16807,2344,3932,8RbwOhM6PUk,so while making my latest for the about seeing...
...,...,...,...,...,...,...,...,...,...,...,...
1734,vaccines,US: Anti-vaccination groups blamed in Minnesot...,US: Anti-vaccination groups blamed in Minnesot...,https://www.youtube.com/watch?v=U8K9bbMo-M4,0,6475,71,29,91,U8K9bbMo-M4,at this Minneapolis Medical Clinic signs in En...
1735,vaccines,"Dr. Anthony Fauci, director of the National In...",Anti-Vaccination: An Extreme Public Health Ris...,https://www.youtube.com/watch?v=duOLIS0Pkew,0,2852,30,13,11,duOLIS0Pkew,>>> SHOULD PARENTS HAVE THEIR CHILDREN VAC...
1736,vaccines,Parents in one Oregon town share with CNN's Ga...,Parent: 'Too many' vaccinations recommended fo...,https://www.youtube.com/watch?v=u8lVmtYHNL4,1,10883,72,67,331,u8lVmtYHNL4,RECEIVE AND WHEN THEY SHOULD GET THEM. >> Rep...
1737,vaccines,What is the role of gov't?\nWatch Andrea Tanta...,Debate heats up over vaccine exemptions,https://www.youtube.com/watch?v=_tc6lD6XTaI,1,2083,14,5,7,_tc6lD6XTaI,starting today kids in Oregon can be sent home...


In [7]:
num_neutral = 0
num_promote = 0
num_debunk = 0

for idx in finished_df.index:
    if finished_df['normalized_annotation'][idx] == 1:
        num_neutral += 1
    elif finished_df['normalized_annotation'][idx] == 2:
        num_debunk += 1
    elif finished_df['normalized_annotation'][idx] == 0:
        num_promote += 1

    
print('Neutral: ' + str(num_neutral) + '\nPromote misinfo: ' + str(num_promote) + '\nDebunk misinfo: ' + str(num_debunk))

Neutral: 1145
Promote misinfo: 340
Debunk misinfo: 212


In [8]:
# exploration
onehot = pd.get_dummies(finished_df['normalized_annotation'])
onehot = onehot.rename({
    0: 'promote',
    1: 'neutral',
    2: 'debunk'
}, axis=1)
onehot

,promote,neutral,debunk
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
1734,True,False,False
1735,True,False,False
1736,False,True,False
1737,False,True,False


In [9]:
finished_df['promote'] = onehot['promote']
finished_df['neutral'] = onehot['neutral']
finished_df['debunk'] = onehot['debunk']
finished_df

C:\Users\david\AppData\Local\Temp\ipykernel_9964\1411453132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finished_df['promote'] = onehot['promote']
C:\Users\david\AppData\Local\Temp\ipykernel_9964\1411453132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finished_df['neutral'] = onehot['neutral']
C:\Users\david\AppData\Local\Temp\ipykernel_9964\1411453132.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,Topic,Description,Title,vid_url,normalized_annotation,Num_of_Views,Likes,Dislikes,Number_of_Comments,Video_ID,Captions,promote,neutral,debunk
0,chemtrails,Texts between Sean Hannity and convicted Trump...,A Judge Releases Private Texts Between Sean Ha...,https://www.youtube.com/watch?v=l8laDBhw6kA,1,510265,4436,681,2532,l8laDBhw6kA,>>> WHY DO YOU THINK NANCY >>> WHY DO YOU THIN...,False,True,False
1,chemtrails,INSTAGRAM FLYWITHCAPTAINJOE: https://goo.gl/TT...,COCKPIT WINDOWS - How to open SLIDING WINDOW a...,https://www.youtube.com/watch?v=XuMtzqtjeEM,1,775342,17572,217,1130,XuMtzqtjeEM,Dear friends and followers welcome back to my ...,False,True,False
2,chemtrails,Mystical BS like this drives me crazy! Living...,living without eating for NINE YEARS' BUSTED,https://www.youtube.com/watch?v=iwOX7vOf0_s,1,2104223,46140,2766,18920,iwOX7vOf0_s,the independent brevarian couple claimed they ...,False,True,False
3,chemtrails,"In part three of Hello World Shenzhen, Bloombe...",Inside China's High-Tech Dystopia,https://www.youtube.com/watch?v=ydPqKhgh9Mg,1,2129195,32969,6255,6828,ydPqKhgh9Mg,There's no disputing that Shenzhen has become ...,False,True,False
4,chemtrails,They claim to have 10x the energy density of a...,New 'Perfect' Battery: BUSTED,https://www.youtube.com/watch?v=8RbwOhM6PUk,1,579669,16807,2344,3932,8RbwOhM6PUk,so while making my latest for the about seeing...,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,vaccines,US: Anti-vaccination groups blamed in Minnesot...,US: Anti-vaccination groups blamed in Minnesot...,https://www.youtube.com/watch?v=U8K9bbMo-M4,0,6475,71,29,91,U8K9bbMo-M4,at this Minneapolis Medical Clinic signs in En...,True,False,False
1735,vaccines,"Dr. Anthony Fauci, director of the National In...",Anti-Vaccination: An Extreme Public Health Ris...,https://www.youtube.com/watch?v=duOLIS0Pkew,0,2852,30,13,11,duOLIS0Pkew,>>> SHOULD PARENTS HAVE THEIR CHILDREN VAC...,True,False,False
1736,vaccines,Parents in one Oregon town share with CNN's Ga...,Parent: 'Too many' vaccinations recommended fo...,https://www.youtube.com/watch?v=u8lVmtYHNL4,1,10883,72,67,331,u8lVmtYHNL4,RECEIVE AND WHEN THEY SHOULD GET THEM. >> Rep...,False,True,False
1737,vaccines,What is the role of gov't?\nWatch Andrea Tanta...,Debate heats up over vaccine exemptions,https://www.youtube.com/watch?v=_tc6lD6XTaI,1,2083,14,5,7,_tc6lD6XTaI,starting today kids in Oregon can be sent home...,False,True,False


In [10]:
y = onehot.values
y

array([[False,  True, False],
       [False,  True, False],
       [False,  True, False],
       ...,
       [False,  True, False],
       [False,  True, False],
       [ True, False, False]])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    finished_df['Captions'], y, test_size=0.15, shuffle=True
)

In [12]:
X_train

737     I got 10 K here for the person who has the hig...
34      hi every very welcome to mentor another video ...
38      Dear Friends and Followers Welcome back to my ...
793     can people hear us are we live yet I believe w...
1192    coming up next on insights on PBS Hawaii shoul...
                              ...                        
1518    the Apple Corporation has produced some of the...
480     welcome climate viewers my name is Jim Lee fro...
116     What started as a platform for hobbyists is po...
1530    I want to first say that what I'm going to tal...
478     what is going on guys welcome back to the cons...
Name: Captions, Length: 1442, dtype: string

In [41]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)

In [44]:
# bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

concat = tf.keras.layers.Concatenate()(outputs)

# nn layers
# l1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
# l1 = tf.keras.layers.Dropout(0.1)(outputs['sequence_output'])
# l2 = tf.keras.layers.LSTM(units=128)(outputs['sequence_output'])
# l4 = tf.keras.layers.LSTM(units=128)(l2)
# l5 = tf.keras.layers.Dropout(0.1)(l2)
out = tf.keras.layers.Dense(units=3)(concat)

KeyError: 0

In [ ]:
# CHANGE OUTPUT LAYER AFTER ADJUSTING 
model = tf.keras.Model(inputs = [text_input], outputs = [out])

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input_3[0][0]']                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(
    loss=loss,
    metrics=['accuracy'],
    optimizer='adam'
)

In [ ]:
y_train = np.array(y_train).astype('int64')

In [ ]:
y_train

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=int64)

In [ ]:
X_train

737     I got 10 K here for the person who has the hig...
34      hi every very welcome to mentor another video ...
38      Dear Friends and Followers Welcome back to my ...
793     can people hear us are we live yet I believe w...
1192    coming up next on insights on PBS Hawaii shoul...
                              ...                        
1518    the Apple Corporation has produced some of the...
480     welcome climate viewers my name is Jim Lee fro...
116     What started as a platform for hobbyists is po...
1530    I want to first say that what I'm going to tal...
478     what is going on guys welcome back to the cons...
Name: Captions, Length: 1442, dtype: string

In [ ]:
history = model.fit(X_train, y_train)

46/46 [==============================] - 118s 3s/step - loss: 0.8012 - accuracy: 0.6699


In [ ]:
# let's try a transformer
transformer_layer = tf.